In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [3]:
#importing Libraries
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import folium


In [43]:
pip install geocoder

     |████████████████████████████████| 98 kB 667 kB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [44]:
import geocoder

# Part 1

In [21]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(link,header=0)[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [22]:
# Removing rows with df.borough == not assigned
# Resetting the index after the deletion

df_can = df[df.Borough != 'Not assigned']
df_can = df_can.reset_index(drop=True)
df_can.shape

(103, 3)

In [23]:
df_can.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


To Check the counts of each postal code 

In [25]:
count = pd.DataFrame(df_can['Postal Code'].value_counts())

In [26]:
count 

,Postal Code
M5B,1
M4L,1
M6R,1
M9L,1
M3N,1
...,...
M8Y,1
M2L,1
M6A,1
M4A,1


In [27]:
count.rename(columns={'Postal Code':'Count'},inplace=True)
count.head()

,Count
M5B,1
M4L,1
M6R,1
M9L,1
M3N,1


In [28]:
for x in count['Count']:
    if x == 2:
        print(True)

From the above test in dataframe 'Count' we confirm that there are no duplicate postal codes in dataframe df_can 

In [36]:
# Checking if col 'Neighborhood' has any value == NOT ASSIGNED
for value in df_can['Neighborhood']:
    if value =='Not assigned':
        print(value)
    

Another way to find a particular value in  a dataframe

In [39]:
df_can.loc[df_can['Neighborhood']=='Not assigned']

,Postal Code,Borough,Neighborhood


In [38]:
count.loc[count['Count']==2]

,Count


In [40]:
# Printing out the shape of the final dataframe 
df_can.shape

(103, 3)

# Part 2

Getting the latitude and longitude of each postal code 

In [56]:
#creating a new df of coordinates 
path= '/home/simran/Downloads/Geospatial_Coordinates.csv'

coord = pd.read_csv(path)
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [57]:
# Merging the 2 dataframes 
df_can = df_can.merge(coord, on = 'Postal Code')
df_can.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Checking if any null value exists

In [61]:
df_can['Latitude'].isnull().value_counts()

False    103
Name: Latitude, dtype: int64

In [62]:
df_can['Longitude'].isnull().value_counts()

False    103
Name: Longitude, dtype: int64

In [65]:
df_can.shape

(103, 5)

# Part 3

Visualizing the Data

In [78]:
can_map = folium.Map(location =[43.651070 , -79.347015],zoom_start=11)

for lat,lon,borough,neigh in zip(df_can['Latitude'],df_can['Longitude'],df_can['Borough'],df_can['Neighborhood']):
    
    label = '{},{}'.format(neigh,borough)
    label = folium.Popup(label,parse_html=True)
    
    folium.CircleMarker([lat,lon],
                       radius = 5,
                       popup=label,
                       color='blue',
                       fill_color = '#3186cc',
                        fill=True,
                        fill_opacity=0.6,
                        parse_html=False
                       ).add_to(can_map)
    
can_map

Most of the neighborhoods are clustered around Toronto, so lets explore it.

In [87]:
# Getting lat,lon of toronto 
address = 'Toronto'
geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

43.6534817 -79.3839347


In [112]:
#define the url components

client_id = 'FX3VCRT3ZNFFR13BF4WJREOU0TYHU1UEV4E5HMGUGI53H5IW'
client_secret = 'QSNVXXMDA5AFZ54JM4PPX42D1RNMY4WRMUIA04ZV3DMFEFJN'
limit = 20

version = '20180605'

In [113]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(client_id,client_secret,version,latitude,longitude,limit)

In [114]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef23de2660a9f001b48eb74'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 600,
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 119,
  'suggestedBounds': {'ne': {'lat': 43.6562956, 'lng': -79.3780615537574},
   'sw': {'lat': 43.650320400000005, 'lng': -79.3878700878797}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.6532

In [115]:
items = results['response']['groups'][0]['items']
items

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5227bb01498e17bf485e6202',
   'name': 'Downtown Toronto',
   'location': {'lat': 43.65323167517444,
    'lng': -79.38529600606677,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.65323167517444,
      'lng': -79.38529600606677}],
    'distance': 113,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto ON', 'Canada']},
   'categories': [{'id': '4f2a25ac4b909258e854f55f',
     'name': 'Neighborhood',
     'pluralName': 'Neighborhoods',
     'shortName': 'Neighborhood',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-5227bb01498e17bf485e6202-0'},
 {'reasons': {'count': 0,
   'items': [{'summar

In [116]:
# Create a new Dataframe of toronto locations

col_names = ['ID','Name','Latitude','Longitude']
places = pd.DataFrame(columns = col_names)
places

,ID,Name,Latitude,Longitude


In [117]:
# loop through the json file and add to the df

for data in items:
    
    Id = data['venue']['id']
    name = data['venue']['name']
    latitude = data['venue']['location']['lat']
    longitude = data['venue']['location']['lng']
    
    places = places.append({
        'ID':Id,
        'Name':name,
        'Latitude':latitude,
        'Longitude':longitude
    },ignore_index=True)

In [118]:
places.head()

,ID,Name,Latitude,Longitude
0,5227bb01498e17bf485e6202,Downtown Toronto,43.653232,-79.385296
1,4ad4c05ef964a520a6f620e3,Nathan Phillips Square,43.652270,-79.383516
2,4e2284b11fc7c0ef9857d143,Chatime 日出茶太,43.655542,-79.384684
3,4b2a6eb8f964a52012a924e3,Indigo,43.653515,-79.380696
4,4ad4c05ef964a520e2f620e3,Textile Museum of Canada,43.654396,-79.386500


In [119]:
places.shape

(20, 4)

In [121]:
toronto = folium.Map(location =[43.6534817, -79.3839347],zoom_start=16)

for lat,lon,label in zip(places['Latitude'],places['Longitude'],places['Name']):
    
    label = '{}'.format(label)
    label = folium.Popup(label,parse_html=True)
    
    folium.CircleMarker([lat,lon],
                       radius = 5,
                       popup=label,
                       color='red',
                       fill_color = 'red',
                        fill=True,
                        fill_opacity=0.2,
                        parse_html=False
                       ).add_to(toronto)
toronto

In [133]:
new = df_can['Borough'].value_counts()
new

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

Visualizing North York

In [134]:
# Getting lat,lon of toronto 
address = 'North York'
geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

43.7543263 -79.44911696639593


In [140]:
df_north = df_can[df_can['Borough']=='North York']
df_north = df_north.reset_index(drop=True)
df_north

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,Don Mills,43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


In [147]:
north = folium.Map(location =[latitude, longitude],zoom_start=12)

for lat,lon,neigh in zip(df_north['Latitude'],df_north['Longitude'],df_north['Neighborhood']):
    
    label = '{}'.format(neigh)
    label = folium.Popup(label,parse_html=True)
    
    folium.CircleMarker([lat,lon],
                       radius = 5,
                       popup=label,
                       color='blue',
                       fill_color = '#3186cc',
                        fill=True,
                        fill_opacity=0.6,
                        parse_html=False
                       ).add_to(north)
    
north

In [142]:
#define the url components

client_id = 'FX3VCRT3ZNFFR13BF4WJREOU0TYHU1UEV4E5HMGUGI53H5IW'
client_secret = 'QSNVXXMDA5AFZ54JM4PPX42D1RNMY4WRMUIA04ZV3DMFEFJN'
limit = 20

version = '20180605'

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(client_id,client_secret,version,latitude,longitude,limit)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef24df760ba080024ab607d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 3318,
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 133,
  'suggestedBounds': {'ne': {'lat': 43.76661198703175,
    'lng': -79.41769376857701},
   'sw': {'lat': 43.724123191093724, 'lng': -79.47702865511644}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a888f7647f8767d37b92f00',
       'name': 'Grill Gate',
       'location': {'address': '832 Sheppard West',
        'crossStreet': 'NE corner of Sheppard & Wilmington',
        'lat': 43.753122978404804,
        'lng': -79.45169001817703,
   

In [143]:
# Create a new Dataframe of North York locations

col_names = ['ID','Name','Latitude','Longitude']
north_places = pd.DataFrame(columns = col_names)
items = results['response']['groups'][0]['items']

# loop through the json file and add to the df

for data in items:
    
    Id = data['venue']['id']
    name = data['venue']['name']
    latitude = data['venue']['location']['lat']
    longitude = data['venue']['location']['lng']
    
    north_places = north_places.append({
        'ID':Id,
        'Name':name,
        'Latitude':latitude,
        'Longitude':longitude
    },ignore_index=True)

In [144]:
north_places.head()

,ID,Name,Latitude,Longitude
0,5a888f7647f8767d37b92f00,Grill Gate,43.753123,-79.451690
1,4b0056b3f964a520013d22e3,Earl Bales Park,43.753043,-79.436228
2,4adf35f1f964a520577821e3,Wolfie's Deli,43.754875,-79.442438
3,4ce01e0294c3b60c0a816fea,Best for Bride in Toronto,43.755789,-79.437834
4,5574bc8b498edad3a0cb6404,Archers Arena,43.754101,-79.467200


In [148]:
for lat,lon,label in zip(north_places['Latitude'],north_places['Longitude'],north_places['Name']):
    
    label = '{}'.format(label)
    label = folium.Popup(label,parse_html=True)
    
    folium.CircleMarker([lat,lon],
                       radius = 5,
                       popup=label,
                       color='green',
                       fill_color = 'green',
                        fill=True,
                        fill_opacity=0.2,
                        parse_html=False
                       ).add_to(north)
north

From this we can see that popular locations are situated near the airport

In [150]:
venue = north_places.loc[0,:]
venue

ID           5a888f7647f8767d37b92f00
Name                       Grill Gate
Latitude                      43.7531
Longitude                    -79.4517
Name: 0, dtype: object

In [151]:
#define the url components

client_id = 'FX3VCRT3ZNFFR13BF4WJREOU0TYHU1UEV4E5HMGUGI53H5IW'
client_secret = 'QSNVXXMDA5AFZ54JM4PPX42D1RNMY4WRMUIA04ZV3DMFEFJN'
limit = 20
venue_id = '5a888f7647f8767d37b92f00'
version = '20180605'
latitude = '43.7531'
longitude = '-79.4517'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(venue_id,client_id,client_secret,version,latitude,longitude,limit)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef251621187ee001bb11700'},
 'response': {'venue': {'id': '5a888f7647f8767d37b92f00',
   'name': 'Grill Gate',
   'contact': {'phone': '+6478085915', 'formattedPhone': '+64 7-808 5915'},
   'location': {'address': '832 Sheppard West',
    'crossStreet': 'NE corner of Sheppard & Wilmington',
    'lat': 43.753122978404804,
    'lng': -79.45169001817703,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.753122978404804,
      'lng': -79.45169001817703}],
    'distance': 2,
    'postalCode': 'M3H 2T1',
    'cc': 'CA',
    'neighborhood': 'Bathurst Manor',
    'city': 'North York',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['832 Sheppard West (NE corner of Sheppard & Wilmington)',
     'North York ON M3H 2T1',
     'Canada']},
   'canonicalUrl': 'https://foursquare.com/v/grill-gate/5a888f7647f8767d37b92f00',
   'categories': [{'id': '4bf58dd8d48988d1c0941735',
     'name': 'Mediterranean Restaurant',
     'pluralName':

Get Over all rating of Grill Gate

In [152]:
try:
    print(results['response']['venue']['rating'])
except:
    print('This venue has no rating')

8.5


Getting the overall rating of all venues in North York

In [160]:
col = ['Name','Rating','Tip counts']
df_rating = pd.DataFrame(columns=col)
for Id in north_places['ID']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(Id,client_id,client_secret,version,latitude,longitude,limit)
    results = requests.get(url).json()
    rating = results['response']['venue']['rating']
    tips = results['response']['venue']['tips']['count']
    name = results['response']['venue']['name']
    df_rating = df_rating.append({'Name':name, 'Rating':rating,'Tip counts':tips},ignore_index=True)

In [161]:
df_rating

,Name,Rating,Tip counts
0,Grill Gate,8.5,3
1,Earl Bales Park,9.2,12
2,Wolfie's Deli,8.2,3
3,Best for Bride in Toronto,8.4,9
4,Archers Arena,8.6,4
5,Le Montmartre,7.8,6
6,Starbucks,8.3,6
7,Earl Bales Ski and Snowboard Centre,8.2,5
8,Baba Ganoush Shawarma Restaurant,8.2,7
9,Pancer's Original Deli,8.2,3


In [169]:
df_rating = df_rating.sort_values(by = ['Rating'],ascending = False,ignore_index=True)
df_rating

,Name,Rating,Tip counts
0,Earl Bales Park,9.2,12
1,Nordstrom,8.9,0
2,Crate & Barrel,8.8,3
3,Aroma Espresso Bar,8.7,9
4,LCBO,8.6,6
5,Archers Arena,8.6,4
6,Grill Gate,8.5,3
7,Best for Bride in Toronto,8.4,9
8,Starbucks,8.3,6
9,True North Climbing,8.3,6


The above DataFrame consist of all venues near North York along with its rating and number of tips 